In [1]:
!pip install huggingface_hub
!pip install transformers
!pip install accelerate  peft  bitsandbytes  trl
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 7.0 MB/s eta 0:00:00


In [2]:
pip install git+https://github.com/huggingface/transformers


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-366is50t
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-366is50t
  Resolved https://github.com/huggingface/transformers to commit 641adca55832ed9c5648f54dcd8926d67d3511db
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.32.0.dev0-py3-none-any.whl size=7450897 sha256=0caa7e8e6d1e9b8ff5c93de5ed6b199e48b0356f2369514d8624fb7eb16af841
  Stored in directory: /tmp/pip-ephem-wheel-cache-1aw69pm2/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2
Note: you may need to restart the kernel

In [3]:
pip install accelerate bitsandbytes


Note: you may need to restart the kernel to use updated packages.


In [4]:
from peft import PeftModel, get_peft_model, LoraConfig, TaskType, prepare_model_for_int8_training
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
def load_peft_model(model,peft_model):
    peft_model = PeftModel.from_pretrained(model, peft_model)
    #merged_model = peft_model.merge_and_unload()
    return peft_model 

In [6]:
model_id = "daryl149/llama-2-7b-chat-hf"
peft_model = '/kaggle/input/llama-output'
model = LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)     # load pretrained model.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
model = load_peft_model(model, peft_model)      # load lora weight.

tokenizer = LlamaTokenizer.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [8]:
eval_prompt = "### Question: whats the best cryptocurrency?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

### Question: whats the best cryptocurrency?
 ### Answer: The best cryptocurrency is the one that you believe in and that has the most potential for growth. However, it's important to do your own research and due diligence before investing in any cryptocurrency. Here are some factors to consider when evaluating a cryptocurrency: 1. Market capitalization: The market capitalization of a cryptocurrency is the total value of all of its coins or tokens in circulation. A higher market capitalization generally indicates a more established and successful cryptocurrency. 2. Transaction volume: The transaction volume of a cryptocurrency is the number of transactions that are processed on its network. A higher transaction volume generally indicates a more active and successful cryptocurrency. 3. Security: The security of a cryptocurrency is the level of protection against hacking and other security threats. A cryptocurrency with strong security measures is more likely to be successful in the long

In [9]:
eval_prompt = "### Question: give me the command line to check my own wallet balance?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

### Question: give me the command line to check my own wallet balance?
 ### Answer: You can use the following command to check your own wallet balance: $ bitcoin-cli getbalance


In [10]:
eval_prompt = "### Question: give me the command line to check my own etherium wallet balance?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

### Question: give me the command line to check my own etherium wallet balance?
 ### Answer: You can use the following command to check your Ethereum wallet balance:
`etherscan balance <your-ethereum-address>`
Replace `<your-ethereum-address>` with your Ethereum address.


In [11]:
eval_prompt = "### Question: give me the command line to check my etherium wallet with the addres of '0xb794f5ea0ba39494ce839613fffba74279579268'?\n ### Answer:"

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))

### Question: give me the command line to check my etherium wallet with the addres of '0xb794f5ea0ba39494ce839613fffba74279579268'?
 ### Answer: You can use the command line to check your Ethereum wallet with the address of '0xb794f5ea0ba39494ce839613fffba74279579268' by using the following command: $etherscan -a 0xb794f5ea0ba39494ce839613fffba74279579268 This will display information about the transaction history of the address, including the balance of Ether and any other tokens that are stored in the address.
